In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

%load_ext autoreload
%autoreload 2

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
mimic = [n for n in catalog.list() if n.startswith('mimic.') and not 'raw' in n]
for table in mimic:
    n, d = catalog.load(table).shape
    print(f"{table:>35s}: {n:>11,d} {d}")

                mimic.core_patients:     382,278 5
              mimic.core_admissions:     523,740 14
                         mimic.keys:     382,278 0
                 mimic.icu_icustays:      76,540 7
              mimic.icu_chartevents: 329,499,788 10
               mimic.core_transfers:   2,189,535 6
                 mimic.hosp_d_hcpcs:      89,200 3
           mimic.hosp_diagnoses_icd:   5,280,351 4
         mimic.hosp_d_icd_diagnoses:     109,775 2
        mimic.hosp_d_icd_procedures:      85,257 2
              mimic.hosp_d_labitems:       1,630 4
                mimic.hosp_drgcodes:     769,622 6
             mimic.hosp_hcpcsevents:     160,727 5
      mimic.hosp_microbiologyevents:   3,397,914 23
              mimic.hosp_poe_detail:   3,256,358 4
          mimic.hosp_procedures_icd:     779,625 5
                mimic.hosp_services:     562,892 4
           mimic.icu_datetimeevents:   7,495,712 8
                  mimic.icu_d_items:       3,861 8
             mimic.icu_outpu

In [ ]:
a = catalog.load('mimic.icu_chartevents').sample()

In [ ]:
catalog.load('mimic.icu_chartevents').shape

(329499788, 10)


In [ ]:
patients = catalog.load('mimic.core_patients')()
stays = catalog.load('mimic.icu_icustays')()
events = catalog.load('mimic.icu_chartevents').sample()

In [ ]:
events.shape

(61879206, 10)


In [ ]:
import pandas as pd

birth_year = patients["anchor_year"] - patients["anchor_age"]
birth_year_date = pd.to_datetime(birth_year, format="%Y")

test = (
    events.join(
        stays.drop(columns=["subject_id", "hadm_id"]), how="inner", on="stay_id"
    )
    .join(
        patients.drop(
            columns=["anchor_age", "anchor_year", "anchor_year_group"]
        ).assign(birth_year=birth_year_date),
        how="inner",
        on="subject_id",
    )
    .drop(columns=["hadm_id", "stay_id"])
)
test.head()


,subject_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,first_careunit,last_careunit,intime,outtime,los,gender,dod,birth_year
175171303,17498523,2169-08-14 18:30:00,2169-08-14 23:57:00,220734,6.5,6.500,units,False,Trauma SICU (TSICU),Trauma SICU (TSICU),2169-08-14 19:56:00,2169-08-15 21:33:28,1.067685,F,NaT,2095-01-01
175171304,17498523,2169-08-14 18:30:00,2169-08-14 23:57:00,227471,1.009,1.009,NaN,False,Trauma SICU (TSICU),Trauma SICU (TSICU),2169-08-14 19:56:00,2169-08-15 21:33:28,1.067685,F,NaT,2095-01-01
175171305,17498523,2169-08-14 20:14:00,2169-08-14 21:02:00,220277,98,98.000,%,False,Trauma SICU (TSICU),Trauma SICU (TSICU),2169-08-14 19:56:00,2169-08-15 21:33:28,1.067685,F,NaT,2095-01-01
175171306,17498523,2169-08-14 20:15:00,2169-08-14 20:19:00,220046,135,135.000,bpm,False,Trauma SICU (TSICU),Trauma SICU (TSICU),2169-08-14 19:56:00,2169-08-15 21:33:28,1.067685,F,NaT,2095-01-01
175171307,17498523,2169-08-14 20:15:00,2169-08-14 20:19:00,220047,70,70.000,bpm,False,Trauma SICU (TSICU),Trauma SICU (TSICU),2169-08-14 19:56:00,2169-08-15 21:33:28,1.067685,F,NaT,2095-01-01


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61879206 entries, 175171303 to 287546602
Data columns (total 16 columns):
 #   Column          Dtype         
---  ------          -----         
 0   subject_id      int32         
 1   charttime       datetime64[ns]
 2   storetime       datetime64[ns]
 3   itemid          category      
 4   value           category      
 5   valuenum        float32       
 6   valueuom        category      
 7   warning         bool          
 8   first_careunit  category      
 9   last_careunit   category      
 10  intime          datetime64[ns]
 11  outtime         datetime64[ns]
 12  los             float64       
 13  gender          category      
 14  dod             datetime64[ns]
 15  birth_year      datetime64[ns]
dtypes: bool(1), category(6), datetime64[ns](6), float32(1), float64(1), int32(1)
memory usage: 4.7 GB


In [ ]:
import numpy as np

np.count_nonzero(pd.isna(test['intime']))

0


In [ ]:
from pasteur.extras.views.mimic import MimicBillion

In [ ]:
b = MimicBillion()
funs = b.ingest('table', **{n: catalog.load(f"mimic.{n}") for n in b.deps['table']})

In [ ]:
next(iter(funs))()


{
    '3_0_0':            subject_id           charttime           storetime  itemid  \
175171303    17498523 2169-08-14 18:30:00 2169-08-14 23:57:00  220734   
175171304    17498523 2169-08-14 18:30:00 2169-08-14 23:57:00  227471   
175171305    17498523 2169-08-14 20:14:00 2169-08-14 21:02:00  220277   
175171306    17498523 2169-08-14 20:15:00 2169-08-14 20:19:00  220046   
175171307    17498523 2169-08-14 20:15:00 2169-08-14 20:19:00  220047   
...               ...                 ...                 ...     ...   
222629546    11382535 2187-10-16 16:00:00 2187-10-16 16:24:00  224016   
222629547    11382535 2187-10-16 16:00:00 2187-10-16 16:24:00  224017   
222629548    11382535 2187-10-16 16:00:00 2187-10-16 16:25:00  224026   
222629549    11382535 2187-10-16 16:00:00 2187-10-16 16:25:00  224027   
222629550    11382535 2187-10-16 16:00:00 2187-10-16 16:25:00  224028   

                  value  valuenum valueuom  warning            first_careunit  \
175171303           6.5   

In [ ]:
from pasteur.extras.utils import ColumnResampler

In [ ]:
r = ColumnResampler(a['value'], 375, True)